In [1]:
#!pip install -r requirements.txt

In [2]:
#!pip3 install ffmpeg

In [3]:
#!pip3 install --upgrade tensorflow keras numpy pandas sklearn pillow

In [6]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from data import DataSet
import time
import os.path
import shutil
import csv
import glob
import os
import os.path
from subprocess import call

In [7]:
def get_train_test_lists(version='01'):
    """
    Using one of the train/test files (01, 02, or 03), get the filename
    breakdowns we'll later use to move everything.
    """
    # Get our files based on version.
    test_file = os.path.join('/home/shared/workspace/rose_ntu_dataset/'+'ntutraintest_rgb', 'testlist' + version + '.txt')
    train_file = os.path.join('/home/shared/workspace/rose_ntu_dataset/'+'ntutraintest_rgb', 'trainlist' + version + '.txt')

    # Build the test list.
    with open(test_file) as fin:
        test_list = [row.strip() for row in list(fin)]

    # Build the train list. Extra step to remove the class index.
    with open(train_file) as fin:
        train_list = [row.strip() for row in list(fin)]
        train_list = [row.split(' ')[0] for row in train_list]

    # Set the groups in a dictionary.
    file_groups = {
        'train': train_list,
        'test': test_list
    }
    print(file_groups['train'],file_groups['test'])
    return file_groups

In [8]:
file_groups = get_train_test_lists()

['sneezeCough/S010C003P019R002A041_rgb', 'sneezeCough/S005C001P021R001A041_rgb', 'sneezeCough/S003C002P001R001A041_rgb', 'sneezeCough/S005C001P013R002A041_rgb', 'sneezeCough/S015C001P025R001A041_rgb', 'sneezeCough/S013C002P037R002A041_rgb', 'sneezeCough/S012C003P028R001A041_rgb', 'sneezeCough/S011C003P025R002A041_rgb', 'sneezeCough/S001C003P007R002A041_rgb', 'sneezeCough/S012C003P019R002A041_rgb', 'sneezeCough/S017C001P015R002A041_rgb', 'sneezeCough/S017C002P003R002A041_rgb', 'sneezeCough/S008C001P034R001A041_rgb', 'sneezeCough/S008C002P015R002A041_rgb', 'sneezeCough/S013C002P017R002A041_rgb', 'sneezeCough/S001C002P003R002A041_rgb', 'sneezeCough/S011C001P008R002A041_rgb', 'sneezeCough/S009C002P025R002A041_rgb', 'sneezeCough/S016C003P019R001A041_rgb', 'sneezeCough/S012C002P028R002A041_rgb', 'sneezeCough/S015C001P015R001A041_rgb', 'sneezeCough/S006C001P015R001A041_rgb', 'sneezeCough/S007C001P028R001A041_rgb', 'sneezeCough/S010C003P007R002A041_rgb', 'sneezeCough/S014C003P019R001A041_rgb',

In [ ]:
# for group, videos in file_groups.items():
#      for video in videos:
#             # Get the parts.
#             print(video)
#             parts = video.split(os.path.sep)
#             classname = parts[0]
#             filename = parts[1].split(" ")[0]

In [ ]:
# print(classname)

In [ ]:
# if not os.path.exists(os.path.join('/home/shared/workspace/rose_ntu_dataset/C3D/', group, classname)):
#     print("Creating folder for %s/%s" % (group, classname))
#     os.makedirs(os.path.join('/home/shared/workspace/rose_ntu_dataset/C3D/'+group, classname))

In [ ]:
# class_file = '/home/shared/workspace/rose_ntu_dataset/nturgb/'+ classname+'/'+filename+'.avi'
# print(class_file)
# if not os.path.exists(class_file):
#     print("/home/shared/workspace/rose_ntu_dataset/nturgb"+classname+'/'+filename)
#     print("Can't find %s to move. Skipping." % (class_dir))
        
# dest = os.path.join("/home/shared/workspace/rose_ntu_dataset/C3D/",group, classname)
# print("Copying %s to %s" % (class_file, dest))
# shutil.copy(class_file, dest)

In [9]:
def move_files(file_groups):
    """This assumes all of our files are currently in _this_ directory.
    So move them to the appropriate spot. Only needs to happen once.
    """
    # Do each of our groups.
    for group, videos in file_groups.items():
        
        # Do each of our videos.
        for video in videos:

            # Get the parts.
            parts = video.split(os.path.sep)
            classname = parts[0]
            filename = parts[1].split(" ")[0]

            # Check if this class exists.
            if not os.path.exists(os.path.join('/home/shared/workspace/rose_ntu_dataset/C3D/'+ group, classname)):
                print("Creating folder for %s/%s" % (group, classname))
                os.makedirs(os.path.join('/home/shared/workspace/rose_ntu_dataset/C3D/'+group, classname))

            # Check if we have already moved this file, or at least that it
            # exists to move.
            class_file = '/home/shared/workspace/rose_ntu_dataset/nturgb/'+ classname+'/'+filename+'.avi'
            if not os.path.exists(class_file):
                print("Can't find %s to move. Skipping." % (class_file))
                continue

            # Move it.
            dest = os.path.join("/home/shared/workspace/rose_ntu_dataset/C3D/",group, classname)
            print("Moving %s to %s" % (filename, dest))
            print("Copying %s to %s" % (class_file, dest))
            shutil.copy(class_file, dest)

In [10]:
# move_files(file_groups)

In [11]:
pwd

'/home/shared/workspace/five-video/five-video-classification-methods'

In [12]:
#cd data

In [13]:
def extract_files():
    """After we have all of our videos split between train and test, and
    all nested within folders representing their classes, we need to
    make a data file that we can reference when training our RNN(s).
    This will let us keep track of image sequences and other parts
    of the training process.

    We'll first need to extract images from each of the videos. We'll
    need to record the following data in the file:

    [train|test], class, filename, nb frames

    Extracting can be done with ffmpeg:
    `ffmpeg -i video.mpg image-%04d.jpg`
    """
    data_file = []
    folders = ['train', 'test']

    for folder in folders:
        class_folders = glob.glob(os.path.join(folder, '*'))
        print(class_folders)
        for vid_class in class_folders:
            class_files = glob.glob(os.path.join(vid_class, '*.avi'))
            print(class_files)
            for video_path in class_files:
                # Get the parts of the file.
                video_parts = get_video_parts(video_path)

                train_or_test, classname, filename_no_ext, filename = video_parts
                print(train_or_test, classname, filename_no_ext, filename)

                # Only extract if we haven't done it yet. Otherwise, just get
                # the info.
                if not check_already_extracted(video_parts):
                    # Now extract it.
                    src = os.path.join(train_or_test, classname, filename)
                    dest = os.path.join(train_or_test, classname,
                        filename_no_ext + '-%04d.jpg')
                    call(["ffmpeg", "-i", src, dest])

                # Now get how many frames it is.
                nb_frames = get_nb_frames_for_video(video_parts)

                data_file.append([train_or_test, classname, filename_no_ext, nb_frames])

                print("Generated %d frames for %s" % (nb_frames, filename_no_ext))

    with open('data_file.csv', 'w') as fout:
        writer = csv.writer(fout)
        writer.writerows(data_file)

    print("Extracted and wrote %d video files." % (len(data_file)))

In [14]:
def get_video_parts(video_path):
    """Given a full path to a video, return its parts."""
    parts = video_path.split(os.path.sep)
    filename = parts[2]
    filename_no_ext = filename.split('.')[0]
    classname = parts[1]
    train_or_test = parts[0]

    return train_or_test, classname, filename_no_ext, filename

In [15]:
def check_already_extracted(video_parts):
    """Check to see if we created the -0001 frame of this file."""
    train_or_test, classname, filename_no_ext, _ = video_parts
    return bool(os.path.exists(os.path.join(train_or_test, classname,
                               filename_no_ext + '-0001.jpg')))

In [16]:
def get_nb_frames_for_video(video_parts):
    """Given video parts of an (assumed) already extracted video, return
    the number of frames that were extracted."""
    train_or_test, classname, filename_no_ext, _ = video_parts
    generated_files = glob.glob(os.path.join(train_or_test, classname,
                                filename_no_ext + '*.jpg'))
    return len(generated_files)

In [17]:
# extract_files()

In [18]:
pwd

'/home/shared/workspace/five-video/five-video-classification-methods'

In [19]:
def train(data_type, seq_length, model, saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=False, batch_size=32, nb_epoch=100):
    # Helper: Save the model.
    checkpointer = ModelCheckpoint(
        filepath=os.path.join('data', 'checkpoints', model + '-' + data_type + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

    # Helper: TensorBoard
    tb = TensorBoard(log_dir=os.path.join('data', 'logs', model))

    # Helper: Stop when we stop learning.
    early_stopper = EarlyStopping(patience=5)

    # Helper: Save results.
    timestamp = time.time()
    csv_logger = CSVLogger(os.path.join('data', 'logs', model + '-' + 'training-' + \
        str(timestamp) + '.log'))

    # Get the data and process it.
    if image_shape is None:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit
        )
    else:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape
        )

    # Get samples per epoch.
    # Multiply by 0.7 to attempt to guess how much of data.data is the train set.
    steps_per_epoch = (len(data.data) * 0.7) // batch_size

    if load_to_memory:
        # Get data.
        X, y = data.get_all_sequences_in_memory('train', data_type)
        X_test, y_test = data.get_all_sequences_in_memory('test', data_type)
    else:
        # Get generators.
        generator = data.frame_generator(batch_size, 'train', data_type)
        val_generator = data.frame_generator(batch_size, 'test', data_type)

    # Get the model.
    rm = call_c3d(seq_length, len(data.classes))

    # Fit!
    if load_to_memory:
        # Use standard fit.
        rm.fit(
            X,
            y,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger],
            epochs=nb_epoch)
    else:
        # Use fit generator.
        rm.fit(
            generator,
            steps_per_epoch=steps_per_epoch,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger, checkpointer],
            validation_data=val_generator,
            validation_steps=40,
            workers=4)

In [20]:
checkpointer = ModelCheckpoint(
        filepath=os.path.join('data', 'checkpoints', 'c3d' + '-' + 'image' + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

In [21]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam,RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys

In [22]:
def c3d(input_shape, num_classes):
        """
        Build a 3D convolutional network, aka C3D.
            https://arxiv.org/pdf/1412.0767.pdf

        With thanks:
            https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2
        """
        model = Sequential()

        # 1st layer group
        model.add(Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv1a",
                   input_shape=input_shape))

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               padding='valid', name='pool1'))
        # 2nd layer group
        model.add(Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv2a"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool2'))
        # 3rd layer group
        model.add(Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv3a"))

        model.add(Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv3b"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool3'))
        # 4th layer group
        model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv4a"))

        model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv4b"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool4'))

        # 5th layer group
        model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv5a"))

        model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv5b"))

        #model.add(ZeroPadding3D(padding=(0, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool5'))
        model.add(Flatten())

        # FC layers group
        model.add(Dense(4096, activation='relu', name='fc6'))
        #model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu', name='fc7'))
        #model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax', dtype="float32"))

        return model

In [23]:
def shallow_c3d(input_shape, num_classes):
        """
        Build a 3D convolutional network, aka C3D.
            https://arxiv.org/pdf/1412.0767.pdf

        With thanks:
            https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2
        """
        model = Sequential()

        # 1st layer group
        model.add(Conv3D(32, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv1a",
                   input_shape=input_shape))

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               padding='valid', name='pool1'))
        # 2nd layer group
        model.add(Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv2a"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool2'))
        # 3rd layer group
        model.add(Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv3a"))

        model.add(Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv3b"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool3'))
        # 4th layer group
        model.add(Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv4a"))

        model.add(Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv4b"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool4'))

#         # 5th layer group
#         model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv5a"))

#         model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv5b"))

#         #model.add(ZeroPadding3D(padding=(0, 1, 1)))
#         model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
#                                padding='valid', name='pool5'))
        model.add(Flatten())

        # FC layers group
        model.add(Dense(1024, activation='relu', name='fc5'))
        model.add(Dropout(0.5))
        model.add(Dense(1024, activation='relu', name='fc6'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax', dtype="float32"))

        return model

In [24]:
def more_shallow_c3d(input_shape, num_classes):
        """
        Build a 3D convolutional network, aka C3D.
            https://arxiv.org/pdf/1412.0767.pdf

        With thanks:
            https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2
        """
        model = Sequential()

        # 1st layer group
        model.add(Conv3D(32, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv1a",
                   input_shape=input_shape))

        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               padding='valid', name='pool1'))
        # 2nd layer group
        model.add(Conv3D(32, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv2a"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool2'))
        model.add(Dropout(0.25))
        
        # 3rd layer group
        model.add(Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv3a"))

        model.add(Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv3b"))

        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool3'))
        model.add(Dropout(0.25))

        # 4th layer group
#         model.add(Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv4a"))

#         model.add(Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv4b"))

#         model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
#                                padding='valid', name='pool4'))

#         # 5th layer group
#         model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv5a"))

#         model.add(Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1), activation='relu', padding='same', name="conv5b"))

#         #model.add(ZeroPadding3D(padding=(0, 1, 1)))
#         model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
#                                padding='valid', name='pool5'))
        model.add(Flatten())

        # FC layers group
#         model.add(Dense(1024, activation='relu', name='fc6'))
#         model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu', name='fc4'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax', dtype="float32"))

        return model

In [25]:
def call_c3d(seq_length, num_classes):
    input_shape = (seq_length, 80, 80, 3)
#     model = c3d(input_shape, num_classes)
    model = more_shallow_c3d(input_shape, num_classes)
#     model = shallow_c3d(input_shape, num_classes)
    optimizer = Adam(learning_rate =1e-3, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    return model

In [26]:
train('images', 16, 'c3d', saved_model=None,
      class_limit=None, image_shape=(80,80,3),
      load_to_memory=False, batch_size=64, nb_epoch=20)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1a (Conv3D)              (None, 16, 80, 80, 32)    2624      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 40, 40, 32)    0         
_________________________________________________________________
conv2a (Conv3D)              (None, 16, 40, 40, 32)    27680     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 20, 20, 32)     0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 20, 20, 32)     0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 20, 20, 64)     55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 20, 20, 64)     1

UnknownError:  FileNotFoundError: [Errno 2] No such file or directory: 'data/train/neckPain/S006C003P008R002A047_rgb-0046.jpg'
Traceback (most recent call last):

  File "/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 249, in __call__
    ret = func(*args)

  File "/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 892, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/ubuntu/.local/lib/python3.6/site-packages/keras/engine/data_adapter.py", line 822, in wrapped_generator
    for data in generator_fn():

  File "/home/ubuntu/.local/lib/python3.6/site-packages/keras/utils/data_utils.py", line 912, in get
    raise e

  File "/home/ubuntu/.local/lib/python3.6/site-packages/keras/utils/data_utils.py", lin